# Training program for Experiment

## setting path

In [1]:
import sys
import os
from datetime import datetime

sys.path.append(os.path.split(os.getcwd())[0])

def get_timef():
    now = datetime.now()
    return now.strftime("%Y%m%d-%H%M")

print(get_timef())

20190902-1641


# Vocab,datasetのロード

In [2]:
from fast_jtnn import *
from MS_PredictModel import MS_Dataset,MS_Dataset_pickle,dataset_load
import pickle
import torch

VOCAB_FILE = "./MS_vocab.txt"

vocab = [x.strip("\r\n ") for x in open(VOCAB_FILE,"r")]
vocab = Vocab(vocab)

'''
MS_Dataset.QUERY = """select smiles,file_path from massbank where ms_type="MS" and instrument_type="EI-B" and smiles<>'N/A';"""
dataset = MS_Dataset(vocab=vocab,host="localhost",database="chemoinfo",batch_size=20)
'''
train_vali_rate = 0.9

train_dataset, vali_dataset = dataset_load("./massbank.pkl",vocab,20,train_vali_rate)
print("number of train dataset :",len(train_dataset))
print("number of validation dataset :",len(vali_dataset))

('number of train dataset :', 6584)
('number of validation dataset :', 732)


## モデルの作成

In [3]:
from ms_encoder import ms_peak_encoder,ms_peak_encoder_lstm,ms_peak_encoder_cnn
import torch.nn as nn
import torch
hidden_size = 100
latent_size = 56
depthT = 20
depthG = 3

dec_model = JTNNVAE(vocab, hidden_size, latent_size, depthT, depthG).to('cuda')
print dec_model
#enc_model = ms_peak_encoder_lstm(train_dataset.max_spectrum_size,output_size=latent_size,\
#        hidden_size=200,embedding_size=20,num_rnn_layers=2,bidirectional=False,dropout_rate=0.5).to('cuda')
enc_model = ms_peak_encoder_cnn(train_dataset.max_spectrum_size,output_size=latent_size,\
                                 conv1_channel=64,conv2_channel=128,conv_output_channel=256,\
                                 kernel1_width=5,kernel2_width=5,conv_output_width=5,\
                                 hidden_size=200,embedding_size=10,num_rnn_layers=2,bidirectional=False,dropout_rate=0.2).to('cuda')
print enc_model

for param in dec_model.parameters():
    if param.dim() == 1:
        nn.init.constant_(param, 0)
    else:
        nn.init.xavier_normal_(param)
load_model = "./vae_model/model.iter-50000"
dec_model.load_state_dict(torch.load(load_model,map_location='cuda'))

print "Model #Params: %dK" % (sum([x.nelement() for x in dec_model.parameters()]) / 1000,)
print "Model #Params: %dK" % (sum([x.nelement() for x in enc_model.parameters()]) / 1000,)

/home/kusachi/anaconda3/envs/jtvae/lib/python2.7/site-packages/torch/nn/_reduction.py:46: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


JTNNVAE(
  (jtnn): JTNNEncoder(
    (embedding): Embedding(1027, 100)
    (outputNN): Sequential(
      (0): Linear(in_features=200, out_features=100, bias=True)
      (1): ReLU()
    )
    (GRU): GraphGRU(
      (W_z): Linear(in_features=200, out_features=100, bias=True)
      (W_r): Linear(in_features=100, out_features=100, bias=False)
      (U_r): Linear(in_features=100, out_features=100, bias=True)
      (W_h): Linear(in_features=200, out_features=100, bias=True)
    )
  )
  (decoder): JTNNDecoder(
    (embedding): Embedding(1027, 100)
    (W_z): Linear(in_features=200, out_features=100, bias=True)
    (U_r): Linear(in_features=100, out_features=100, bias=False)
    (W_r): Linear(in_features=100, out_features=100, bias=True)
    (W_h): Linear(in_features=200, out_features=100, bias=True)
    (W): Linear(in_features=128, out_features=100, bias=True)
    (U): Linear(in_features=128, out_features=100, bias=True)
    (U_i): Linear(in_features=200, out_features=100, bias=True)
    (W_o)

## setting optimizer

In [4]:
import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler

enc_optimizer = optim.Adam(enc_model.parameters(), lr=1e-3)
dec_optimizer = optim.Adam(dec_model.parameters(), lr=1e-3)
#optimizer = optim.SGD(enc_model.parameters(),lr=100)
enc_scheduler = lr_scheduler.ExponentialLR(enc_optimizer, 0.9)
dec_scheduler = lr_scheduler.ExponentialLR(dec_optimizer, 0.9)
#scheduler.step()

In [5]:
from MS_PredictModel import ms_peak_encoder,MS_Dataset
from torch.autograd import Variable
from tqdm import tqdm
import numpy as np

pbar = None
train_dataset.batch_size = 20
vali_dataset.batch_size = 10

anneal_iter = 7400

word_rate=1.0
topo_rate=1.0
assm_rate=1.0

beta = 0
step_beta = 0.002
kl_anneal_iter = 10 # epoch
max_beta = 1
warmup = 50 # epoch

fine_tunning_warmup=30 # epoch

def training(max_epoch = 100):
    global pbar
    global beta
    total_step = 0
    meters = np.zeros(7)
    vali_meters = np.zeros(6)
    with open("log2.csv","w") as f:
        f.write("epoch,iter.,kl_loss,word,topo,assm,wors_loss,topo_loss,assm_loss,vali word,vali topo,vali assm,vali_word_loss,vali_topo_loss,vali_assm_loss\n")
    for epoch in range(max_epoch):
        if epoch % kl_anneal_iter == 0 and epoch >= warmup:
            beta = min(max_beta, beta + step_beta)
        print("epoch : ",epoch)
        for batch in train_dataset:
            x_batch, x_jtenc_holder, x_mpn_holder, x_jtmpn_holder,x,y = batch
            total_step+=1
            #pbar.update(1)
            x = x.to('cuda')
            y = y.to('cuda')
            
            enc_model.zero_grad()
            dec_model.zero_grad()
            enc_optimizer.zero_grad()
            dec_optimizer.zero_grad()
            
            h,kl_loss = enc_model(x,y,training=True,sample=True)
            tree_vec = h[:,:h.shape[1]/2]
            mol_vec  = h[:,h.shape[1]/2:]
            _, x_tree_mess = dec_model.jtnn(*x_jtenc_holder)
            word_loss, topo_loss, word_acc, topo_acc = dec_model.decoder(x_batch,tree_vec)
            assm_loss, assm_acc = dec_model.assm(x_batch, x_jtmpn_holder, mol_vec , x_tree_mess)
            total_loss = word_loss*word_rate+\
                         topo_loss*topo_rate+\
                         assm_loss*assm_rate+\
                         kl_loss*beta
            total_loss.backward()
            enc_optimizer.step()
            if epoch >= fine_tunning_warmup:
                dec_optimizer.step()
            
            del x,y,h

            meters = meters + np.array([kl_loss.item(),word_acc * 100, topo_acc * 100, assm_acc * 100,word_loss.item(),topo_loss.item(),assm_loss.item()])
            if total_step % 200 == 0:
                vali_total = 0
                for batch in vali_dataset:
                    x_batch, x_jtenc_holder, x_mpn_holder, x_jtmpn_holder,x,y = batch
                    x = x.to('cuda')
                    y = y.to('cuda')
                    with torch.no_grad():
                        h,_ = enc_model(x,y,training=False,sample=False)
                        tree_vec = h[:,:h.shape[1]/2]
                        mol_vec  = h[:,h.shape[1]/2:]
                        _, x_tree_mess = dec_model.jtnn(*x_jtenc_holder)
                        word_loss, topo_loss, word_acc, topo_acc = dec_model.decoder(x_batch,tree_vec)
                        assm_loss, assm_acc = dec_model.assm(x_batch, x_jtmpn_holder, mol_vec , x_tree_mess)
                        vali_meters = vali_meters + np.array([word_acc * 100, topo_acc * 100, assm_acc * 100,word_loss.item(),topo_loss.item(),assm_loss.item()])
                        vali_total += 1    
                    del x,y,h
                    
                meters /= 200
                vali_meters /= vali_total
                print "[%d] , kl_loss %.2f, Word: %.2f, Topo: %.2f, Assm: %.2f vali_Word: %.2f, vali_Topo: %.2f, vali_assm: %.2f, learning rate: %.4f" % \
                    (total_step, meters[0], meters[1], meters[2],meters[3], vali_meters[0],vali_meters[1],vali_meters[2],enc_scheduler.get_lr()[0])                
                with open("log2.csv","a") as f:
                    f.write("%d,%d," % (epoch,total_step))
                    f.write("%.2f,%.2f,%.2f,%.2f,%.2f,%.2f,%.2f," % (meters[0], meters[1], meters[2],meters[3],meters[4],meters[5],meters[6]))
                    f.write("%.2f,%.2f,%.2f,%.2f,%.2f,%.2f\n" % (vali_meters[0],vali_meters[1],vali_meters[2],vali_meters[3],vali_meters[4],vali_meters[5]))
                sys.stdout.flush()
                meters *= 0
                vali_meters *= 0
            if total_step % 200 == 0:
                torch.save(enc_model.state_dict(), "./enc_model" + "/model.iter-" + str(total_step))
                torch.save(dec_model.state_dict(), "./dec_model" + "/model.iter-" + str(total_step))
            #if total_step % anneal_iter == 0:
                #scheduler.step()

#import pdb; pdb.set_trace()
try:
    #if pbar is None:
        #pbar = tqdm()
    if not os.path.exists("./enc_model"):
        os.mkdir("./enc_model")
    if not os.path.exists("./dec_model"):
        os.mkdir("./dec_model")
    training(200)
except RuntimeError as e:
    #if pbar is not None:
        #del pbar
    import traceback
    print(traceback.format_exc())
    #import pdb; pdb.set_trace()
    print(e)



('epoch : ', 0)
(torch.Size([20, 310, 10]),)
(torch.Size([20, 310, 1]),)
(torch.Size([20, 310, 11]),)
(torch.Size([20, 310, 64]),)
(torch.Size([20, 310, 64]),)
(torch.Size([20, 310, 200]),)
(torch.Size([20, 200]),)


/home/kusachi/anaconda3/envs/jtvae/lib/python2.7/site-packages/torch/nn/functional.py:1386: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
/home/kusachi/anaconda3/envs/jtvae/lib/python2.7/site-packages/torch/nn/functional.py:1374: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


[200] , kl_loss 434.00, Word: 34.00, Topo: 85.56, Assm: 83.03 vali_Word: 36.19, vali_Topo: 85.97, vali_assm: 84.42, learning rate: 0.0010
('epoch : ', 1)
[400] , kl_loss 640.46, Word: 35.01, Topo: 85.80, Assm: 84.48 vali_Word: 36.98, vali_Topo: 85.50, vali_assm: 82.29, learning rate: 0.0010
[600] , kl_loss 611.64, Word: 36.28, Topo: 86.00, Assm: 84.54 vali_Word: 38.51, vali_Topo: 86.01, vali_assm: 84.79, learning rate: 0.0010
('epoch : ', 2)
[800] , kl_loss 548.46, Word: 37.72, Topo: 86.18, Assm: 85.43 vali_Word: 40.58, vali_Topo: 85.81, vali_assm: 83.63, learning rate: 0.0010
('epoch : ', 3)
[1000] , kl_loss 529.48, Word: 38.50, Topo: 86.27, Assm: 84.19 vali_Word: 41.99, vali_Topo: 86.21, vali_assm: 83.57, learning rate: 0.0010
[1200] , kl_loss 506.20, Word: 40.65, Topo: 86.41, Assm: 85.02 vali_Word: 40.77, vali_Topo: 85.81, vali_assm: 83.35, learning rate: 0.0010
('epoch : ', 4)
[1400] , kl_loss 480.51, Word: 42.39, Topo: 86.79, Assm: 84.90 vali_Word: 45.08, vali_Topo: 86.34, vali_as

In [20]:
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit import DataStructs

model_path = "./enc_model/model.iter-65800"
if model_path is not None:
    enc_model.load_state_dict(torch.load(model_path,map_location='cuda'))

model_path = "./dec_model/model.iter-65800"
if model_path is not None:
    dec_model.load_state_dict(torch.load(model_path,map_location='cuda'))

train_dataset.batch_size = 20
vali_dataset.batch_size = 10

sample_rate = 1.0

def evaluation():
    ret = []
    with torch.no_grad():
        for batch in vali_dataset:
            x_batch, x_jtenc_holder, x_mpn_holder, x_jtmpn_holder,x,y = batch
            x = x.to('cuda')
            y = y.to('cuda')
            
            h,_ = enc_model(x,y,training=False,sample=True,sample_rate=sample_rate)
            tree_vec = h[:,:h.shape[1]/2]
            mol_vec  = h[:,h.shape[1]/2:]
            for num in range(h.size()[0]):
                
                true_smiles=x_batch[num].smiles
                predict_smiles = dec_model.decode(tree_vec[num].view(1,latent_size/2),mol_vec[num].view(1,latent_size/2),False)
                
                #smilesの正規化
                true_smiles = Chem.MolToSmiles(Chem.MolFromSmiles(true_smiles),True)
                predict_smiles = Chem.MolToSmiles(Chem.MolFromSmiles(predict_smiles),True)
                
                ret.append((true_smiles,predict_smiles,h[num].data))
    return ret
result = evaluation()
print(len(result))

730


In [22]:
import os
from rdkit import Chem
from rdkit.Chem import Draw
from PIL import ImageDraw,ImageFont
from rdkit.Chem import AllChem
from rdkit import DataStructs

from shutil import copyfile
from shutil import make_archive
import zipfile

def _re_smiles(smiles1,smiles2):
    #print(smiles1,smiles2)
    smiles1 = Chem.MolToSmiles(Chem.MolFromSmiles(smiles1),True)
    smiles2 = Chem.MolToSmiles(Chem.MolFromSmiles(smiles2),True)
    return smiles1 == smiles2

def is_structural_isomer(smiles1,smiles2):
    def Molecular_formula(smiles):
        atoms = {}
        mol = Chem.AddHs(Chem.MolFromSmiles(smiles))
        for atom in mol.GetAtoms():
            if not atom.GetSymbol() in atoms:
                atoms[atom.GetSymbol()] = 1
            else:
                atoms[atom.GetSymbol()] += 1
        return atoms
    atoms1 = Molecular_formula(smiles1)
    atoms2 = Molecular_formula(smiles2)
    return atoms1 == atoms2
    
def analyze_result(smiles_list,log_path="evaluation.csv",path="image_list",zipname=None):
    if not os.path.exists(path):
        os.mkdir(path)
    log_path = os.path.join(path,log_path)
    with open(log_path,"w") as f:
        print "Number of data: %d"% (len(smiles_list))
        f.write("Number of data,%d,,\n" % (len(smiles_list)))
        
        match_list = [[i,one[0]] for i,one in enumerate(smiles_list) if _re_smiles(one[0],one[1])]
        print "Number of matching: %d" % (len(match_list))
        f.write("Number of matching,%d,,,\n" % (len(match_list)))
        print(match_list)
    
        isomer_list = [[i,one[0]] for i,one in enumerate(smiles_list) if is_structural_isomer(one[0],one[1]) and [i,one[0]] not in match_list]
        print "Number of matching: %d" % (len(isomer_list))
        f.write("Number of isomer,%d,,,\n" % (len(isomer_list)))
        print(isomer_list)
        
        f.write("true,predict,ECFP-Tanimoto score,MACCS-Tanimoto score\n")
        
    ecfp_score_list = []
    maccs_score_list = []
    for i,smiles in enumerate(smiles_list):
        true_mol = Chem.MolFromSmiles(smiles[0])
        predict_mol = Chem.MolFromSmiles(smiles[1])
        
        true_fingerprint = AllChem.GetMorganFingerprint(true_mol,2)
        predict_fingerprint = AllChem.GetMorganFingerprint(predict_mol,2)
        ECFP_score = DataStructs.TanimotoSimilarity(true_fingerprint,predict_fingerprint)
        
        true_fingerprint = AllChem.GetMACCSKeysFingerprint(true_mol)
        predict_fingerprint = AllChem.GetMACCSKeysFingerprint(predict_mol)
        MACCS_score = DataStructs.TanimotoSimilarity(true_fingerprint,predict_fingerprint)
        
        with open(log_path,"a") as f:
            f.write(smiles[0]+","+smiles[1]+","+str(ECFP_score)+","+str(MACCS_score)+",")
            for num in range(smiles[2].shape[0]):
                f.write(str(smiles[2][num].item()) + ",")
            f.write("\n")
        
        image = Draw.MolsToImage([true_mol,predict_mol])
        draw = ImageDraw.Draw(image)
        font = ImageFont.load_default()
        font.size=40
        draw.text((0,0),str(ECFP_score)+","+str(MACCS_score),(0, 0, 0),font=font)
        image.save(os.path.join(path,"%d.png" % i))
        
    
    copyfile("./log2.csv",os.path.join(path,"log.csv"))
    copyfile("./training.ipynb",os.path.join(path,"Untitled.ipynb"))
    copyfile("./ms_encoder.py",os.path.join(path,"ms_encoder.py"))
    if zipname is None:
        make_archive(path,"zip",root_dir=path)
                
    
analyze_result(result,path=os.path.join("result-{}".format(get_timef())))

Number of data: 730
Number of matching: 290
[[4, 'Cc1ccc(C(=O)OCC(C)C)cc1'], [8, 'O=[N+]([O-])c1ccc(I)cc1'], [9, 'O=P(O)(O)c1ccccc1'], [10, 'CCOC(=O)c1cccc(Cl)c1'], [13, 'c1ccc2nc3ccccc3cc2c1'], [14, 'CN1CC2CC(C1)c1cccc(=O)n1C2'], [15, 'C=C1CCC(C(C)=O)C2C=C(C)CCC12'], [16, 'Nc1ccc(O)c(N)c1'], [20, 'CCC(C(=O)O[Si](C)(C)C)c1ccccc1'], [21, 'CC(C)=CCOC(=O)c1ccccc1'], [25, 'Cc1ccc(O)cc1'], [29, 'CC(=O)c1cccc(Oc2ccccc2)c1'], [32, 'Cc1ccc(CC(=O)O[Si](C)(C)C)cc1'], [34, 'CCCCCCCCC=CCCCCCCCCCCO'], [37, 'COC(=O)c1cccc2nc3ccccc3nc12'], [43, 'Cc1c([N+](=O)[O-])cccc1[N+](=O)[O-]'], [44, 'CC(O)c1ccccc1'], [50, 'CC(C)CN'], [51, 'C[Si](C)(C)OC(=O)c1ccc(O[Si](C)(C)C)c(O[Si](C)(C)C)c1'], [59, 'COCCOCCO'], [62, 'CCOC(=O)C=CC=C(C)CCC=C(C)CCC=C(C)C'], [63, 'O=C(C(=O)c1ccccc1)c1ccccc1'], [64, 'COC(=O)c1cnc2n(c1=O)-c1ccccc1CC2'], [65, 'CC=CC(=O)OC'], [66, 'CC(=O)OCC=C(C)CCC=C(C)C'], [67, 'CCCCCCCC(=O)OC'], [68, 'Cc1cccc(C#N)c1'], [69, 'COc1ccc(C(=O)O[Si](C)(C)C)cc1OC'], [71, 'CS(C)=O'], [72, 'O=C(O)c1cc(C(=O

In [19]:
result[0][2].shape

torch.Size([10, 56])

In [8]:
!mv evaluation.csv ./result-1.0
!mv log2.csv ./result-1.0
!zip -r image.zip ./result-1.0 ./ms_encoder.py ./Untitled.ipynb

updating: image_list/ (stored 0%)
updating: image_list/471.png (deflated 10%)
updating: image_list/506.png (deflated 9%)
updating: image_list/688.png (deflated 7%)
updating: image_list/231.png (deflated 8%)
updating: image_list/65.png (deflated 12%)
updating: image_list/345.png (deflated 8%)
updating: image_list/598.png (deflated 15%)
updating: image_list/526.png (deflated 8%)
updating: image_list/184.png (deflated 6%)
updating: image_list/196.png (deflated 7%)
updating: image_list/477.png (deflated 7%)
updating: image_list/139.png (deflated 16%)
updating: image_list/430.png (deflated 7%)
updating: image_list/186.png (deflated 10%)
updating: image_list/611.png (deflated 12%)
updating: image_list/265.png (deflated 11%)
updating: image_list/497.png (deflated 8%)
updating: image_list/204.png (deflated 12%)
updating: image_list/464.png (deflated 7%)
updating: image_list/34.png (deflated 8%)
updating: image_list/237.png (deflated 7%)
updating: image_list/665.png (deflated 6%)
updating: imag

In [44]:
with open("test_vali_data.pkl","wb") as f:
    pickle.dump(vali_dataset,f)

In [45]:
with open("test_vali_data.pkl","rb") as f:
    vali_dataset = pickle.load(f)